In [ ]:
import re
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Lambda, Input
from keras.optimizers import SGD
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import numpy as np
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline

In [26]:
sdb = pd.read_csv("Shakespeare-plays/Shakespeare_data.csv")

In [27]:
class_names = ['comedy','tragedy','other']

In [28]:
def all_the_plays():
    new_play = "Henry IV"
    play_array = []
    for i in sdb['Play']:
        if i != new_play:
            play_array.append(new_play)
            new_play = i
    return play_array

In [29]:
all_plays = all_the_plays()

In [30]:
def get_text(play_name):
    play = sdb[sdb['Play']==play_name]
    w = play['PlayerLine']
    text = ""
    for i in w:
        text += i
        text += " "
    return text

In [31]:
comedy_training = []
tragedy_training = []
history_training = []

In [32]:
def get_training_text():
    comedy_training.append(get_text("Alls well that ends well"))
    comedy_training.append(get_text("As you like it"))
    comedy_training.append(get_text("Loves Labours Lost"))
    comedy_training.append(get_text("Measure for measure"))
    comedy_training.append(get_text("Merry Wives of Windsor"))
    comedy_training.append(get_text("A Midsummer nights dream"))
    comedy_training.append(get_text("Taming of the Shrew"))
    comedy_training.append(get_text("The Tempest"))
    comedy_training.append(get_text("Two Gentlemen of Verona"))
    comedy_training.append(get_text("A Winters Tale"))
    tragedy_training.append(get_text("Antony and Cleopatra"))
    tragedy_training.append(get_text("Coriolanus"))
    tragedy_training.append(get_text("Hamlet"))
    tragedy_training.append(get_text("Julius Caesar"))
    tragedy_training.append(get_text("macbeth"))
    tragedy_training.append(get_text("Othello"))
    tragedy_training.append(get_text("Timon of Athens"))
    tragedy_training.append(get_text("Titus Andronicus"))
    history_training.append(get_text("Henry IV"))
    history_training.append(get_text("Henry VI Part 1"))
    history_training.append(get_text("Henry VI Part 3"))
    history_training.append(get_text("Henry VIII"))
    history_training.append(get_text("King John"))
    history_training.append(get_text("Richard II"))

In [33]:
comedy_testing = []
tragedy_testing = []
history_testing = []

In [34]:
def get_testing_text():
    comedy_testing.append(get_text("A Comedy of Errors"))
    comedy_testing.append(get_text("Merchant of Venice"))
    comedy_testing.append(get_text("Much Ado about nothing"))
    comedy_testing.append(get_text("Twelfth Night"))
    tragedy_testing.append(get_text("Cymbeline"))
    tragedy_testing.append(get_text("King Lear"))
    tragedy_testing.append(get_text("Romeo and Juliet"))
    tragedy_testing.append(get_text("Troilus and Cressida"))
    history_testing.append(get_text("Henry V"))
    history_testing.append(get_text("Henry VI Part 3"))
    history_testing.append(get_text("Pericles"))
    history_testing.append(get_text("Richard III"))

In [35]:
get_training_text()
get_testing_text()

In [36]:
def get_shakes_text_array(text):
    w = re.split("\W*[^\'\w+\']", text)
    text_array = []
    for i in w:
        text_array.append([i])
    return text_array

In [37]:
# def create_model(max_sequence_len, total_words):
#     input_len = max_sequence_len - 1
#     model = Sequential()
    
#     # Add Input Embedding Layer
#     model.add(Embedding(total_words, 10, input_length=input_len))
    
#     # Add Hidden Layer 1 - LSTM Layer
#     model.add(LSTM(512))
#     model.add(Dropout(0.4))
    
#     # Add Output Layer
#     model.add(Dense(total_words, activation='softmax'))

#     model.compile(loss='categorical_crossentropy', optimizer='adam')
    
#     return model

# model = create_model(max_sequence_len, total_words)
# model.summary()

In [38]:
def word_counter(words):
    count_words = Counter(words)
    total_words = len(words)
    sorted_words = count_words.most_common(total_words)
    return sorted_words

In [39]:
# vocab_to_int = {w:i+1 for i, (w,c) in enumerate(sorted_words)}

In [40]:
def word_encoding(words):
    words_split = words.split('\n')
    words_int = []
    for play in words_split:
        p = [words_int[w] for w in play.split()]
        words_int.append(p)
    return words_int

In [41]:
def encoding(labels_split):
    encoded_labels = []
    for l in labels_split:
        if l == 'comedy':
            encoded_labels.append(0)
        elif l == 'tragedy':
            encoded_labels.append(1)
        else:
            encoded_labels.append(2)
    return encoded_labels

In [42]:
# np.array(encoded_labels) = encoding(labels_split)

In [1]:
def pad_features(plays_int, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's or truncated to the input seq_length.
    '''
    features = np.zeros((len(plays_int), seq_length), dtype = int)
    
    for i, p in enumerate(plays_int):
        play_len = len(p)
        
        if play_len <= seq_length:
            zeroes = list(np.zeros(seq_length-play_len))
            new = zeroes+p
        elif play_len > seq_length:
            new = play[0:seq_length]
        
        features[i,:] = np.array(new)
    
    return features